In [62]:
from typing import List, Optional, Callable
import polars as pl
import pyarrow.parquet as pq
import torch
from torch.utils.data import Dataset


# Create a dataset for the Parquet files

In [63]:
class ParquetDataset(Dataset):
    def __init__(self, file_paths: List[str], logging=True):
        self.file_paths = file_paths
        self.file_row_counts: List[int] = [pq.ParquetFile(f).metadata.num_rows for f in file_paths]
        self.cum_row_counts: List[int] = [sum(self.file_row_counts[:i]) for i in range(len(self.file_row_counts))]

        self.pq_df: Optional[pl.DataFrame] = None
        self.pq_df_idx: Optional[int] = None 

        if logging:
            print("Loaded files with rows:")
            for i, file in enumerate(file_paths):
                print(f"\t{self.file_row_counts[i]} : {file}")

            print(f"{len(self)} total samples." )

    def __len__(self) -> int:
        return sum(self.file_row_counts)

    def __getitem__(self, idx: int) -> torch.Tensor:
        pass

    def get_single_row_with_batching(self, row_idx: int, file_idx: int) -> pl.DataFrame:
        if file_idx != self.pq_df_idx:
            self.load_pq_file(idx=file_idx)

        return self.pq_df.row(row_idx)

    def get_single_row(self, idx: int) -> pl.DataFrame:
        # Identify which file and which row within that file corresponds to idx
        file_idx: int = self.calculate_file_index_from_global_index(idx)
        row_idx: int = idx - self.cum_row_counts[file_idx]
        
        # Load data for the required row
        return self.get_row_from_pq_file(file=self.file_paths[file_idx], row_idx=row_idx)

    def calculate_file_index_from_global_index(self, idx: int) -> int:
        for i, cum_row_count in enumerate(self.cum_row_counts):
            if cum_row_count <= idx:
                return i

        raise ValueError(f"Index {idx} too large for dataset with {len(self)} samples")
                
    @staticmethod
    def get_row_from_pq_file(file: str, row_idx: int):
        return pl.read_parquet(file, row_index_offset=row_idx, n_rows=1, use_pyarrow=False)

    @staticmethod
    def convert_df_row_to_tensor(row: pl.DataFrame) -> torch.Tensor:
        # TODO
        return row
    
    def load_pq_file(self, idx: int) -> None:
        self.pq_df_idx = idx
        self.pq_df = pl.read_parquet(self.file_paths[idx])
    

# Instantiate the dataloader.

In [64]:
def make_train_parquet_path(i: int) -> str:
    return f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id={i}/part-0.parquet"
# Setup the file indices to use.
K_MAX_TRAIN_FILES: int = 10
K_TRAIN_FILES: List[str] = [make_train_parquet_path(i) for i in range(K_MAX_TRAIN_FILES)]
K_TEST_FILES: List[str] = ["/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet/date_id=0/part-0.parquet"]

# K_TRAIN_FILE_INDICES: List[int] = [8]

train_dataset = ParquetDataset(file_paths=K_TRAIN_FILES, logging=True)
# test_dataset = ParquetDataset(file_paths=K_TEST_FILES, logging=True)



Loaded files with rows:
	1944210 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=0/part-0.parquet
	2804247 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=1/part-0.parquet
	3036873 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=2/part-0.parquet
	4016784 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=3/part-0.parquet
	5022952 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=4/part-0.parquet
	5348200 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=5/part-0.parquet
	6203912 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=6/part-0.parquet
	6335560 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=7/part-0.parquet
	6140024 : /kaggle/input/jane-street-real-time-market-da

In [71]:
from functools import wraps
from time import time
import random

def timing(f: Callable, *args, **kwargs) -> Callable:
    @wraps(f)
    def wrap(*args, **kwargs):
        ts = time()
        result = f(*args, **kwargs)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kwargs, te-ts))
        return result
    return wrap

@timing
def iterate_samples_by_row(step_size: int):
    for i in range(0, len(train_dataset), step_size):
        train_dataset.get_single_row(i)

@timing
def iterate_samples_batched(step_size: int):
    for file_idx in range(K_MAX_TRAIN_FILES):
        for row_idx in range(0, train_dataset.file_row_counts[file_idx], step_size):
            train_dataset.get_single_row_with_batching(row_idx=row_idx, file_idx=file_idx)

# A simple batching schemem is definitely the way to go. We can iterate orders of magnitude more samples
# in the same amount of time it would take to do naive random access.
iterate_samples_by_row(step_size=1000000)
iterate_samples_by_row(step_size=100000)
iterate_samples_by_row(step_size=10000)
iterate_samples_batched(step_size=1000)
iterate_samples_batched(step_size=100)
iterate_samples_batched(step_size=10)

func:'iterate_samples_by_row' args:[(), {'step_size': 1000000}] took: 1.1238 sec
func:'iterate_samples_by_row' args:[(), {'step_size': 100000}] took: 12.2215 sec
func:'iterate_samples_by_row' args:[(), {'step_size': 10000}] took: 99.2402 sec
func:'iterate_samples_batched' args:[(), {'step_size': 1000}] took: 11.4278 sec
func:'iterate_samples_batched' args:[(), {'step_size': 100}] took: 16.0547 sec
func:'iterate_samples_batched' args:[(), {'step_size': 10}] took: 44.7318 sec


# Prepare data

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
# Separate features and responders
features = sample_df.filter(regex='^feature_')
responders = sample_df.filter(regex='^responder_')
# Convert to numpy arrays for TensorFlow
X = features.values  # Features for input
y = responders.values  # Responders for output
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
y = np.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)

# Build the Autoencoder Model

In [ ]:
# Define the number of input and output nodes
input_dim = X.shape[1]  # Number of features (79)
output_dim = y.shape[1]  # Number of responders (9)
# Define the model
model = models.Sequential([
    layers.Input(shape=(input_dim,)),  # Input layer
    layers.Dense(64, activation='relu'),  # Encoder
    layers.Dense(32, activation='relu'),  # Bottleneck layer (compression)
    layers.Dense(64, activation='relu'),  # Decoder
    layers.Dense(output_dim, activation='linear')  # Output layer for responders
])
model.compile(optimizer='adam', loss='mse')

# Train Autoencoder Model

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 5
    lr = initial_lr * (drop ** (epoch // epochs_drop))
    return lr
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# Define EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',    # Monitor validation loss
    patience=10,            # Number of epochs to wait for improvement
    min_delta=0.001,       # Minimum change to qualify as an improvement
    restore_best_weights=True  # Restore weights from the best epoch
)

history = model.fit(
    X, y,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, lr_scheduler]
)

In [ ]:
model.save("/kaggle/working/model.keras")

# Submission

See [Jane Street RMF Demo Submission](https://www.kaggle.com/code/ryanholbrook/jane-street-rmf-demo-submission) for details

In [ ]:
import os
import polars as pl
import kaggle_evaluation.jane_street_inference_server

In [ ]:
import polars as pl
import numpy as np
# Assuming `model` is your trained model
# Assuming features required by the model are named 'feature_00', 'feature_01', etc.
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    global lags_
    if lags is not None:
        lags_ = lags
    # Extract the features for the model input
    feature_columns = [col for col in test.columns if col.startswith("feature_")]
    features = test.select(feature_columns).to_numpy()  # Convert to numpy array for model input
    features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
    # Generate predictions using the model
    model_predictions = model.predict(features)
    responder_6_predictions = model_predictions[:, 6]  # Assuming responder_6 is at index 6
    # Create a new Polars DataFrame with row_id and responder_6 predictions
    predictions = test.select("row_id").with_columns(
        pl.Series("responder_6", responder_6_predictions)
    )
    # Ensure the output format and length requirements
    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')
    
    assert len(predictions) == len(test)
    return predictions

In [ ]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )